In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import year,month,hour, when, col, date_format, to_timestamp, round, coalesce
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
sc = pyspark.SparkSession('local', 'test')
#sc = pyspark.SparkContext(appName="Data Extraction and manipulation")
sqlContext = SQLContext(sc)

C:\Users\baris\anaconda3\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
df = sqlContext.read.option("header",True).option("inferSchema",True) \
     .csv("pollution_us_2000_2016.csv")

In [4]:
df=df.withColumn("year",year("Date Local")).withColumn("month",month("Date Local"))  

In [5]:
dfmonth=df.select("State","County","year","month","NO2 Mean")\
.groupBy("State","County","year","month").avg("NO2 Mean")
#not used

In [1]:
#dfmonth.toPandas().head(10)

In [40]:
df1=df.select("State","County","year","NO2 Mean","O3 Mean","SO2 Mean")\
.groupBy("State","County","year").avg("NO2 Mean","O3 Mean","SO2 Mean")

In [43]:
df1p=df1.toPandas()
#df1p.to_csv('df1.csv')

In [2]:
#df1p.head(10)

In [45]:
df2p=pd.read_csv("death_data.csv")

In [46]:
df_all=df1p.sort_values(by=['year'])

In [47]:
df2p['State']=df2p['State'].map({'AK':'Alaska','AL':'Alabama','AR':'Arkansas','AZ':'Arizona','CA': 'California','CO':'Colorado',\
                                 'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','IA':'Iowa',\
                                 'ID':'Idaho','IL': 'Illinois','IN':'Indiana','KS':'Kansas','KY':'Kentucky','LA':'Louisiana', \
                                'MA':'Massachusetts','MD':'Maryland','ME':'Maine','MI':'Michigan','MN':'Minnesota','MO':'Missouri', \
                                'NC':'North Carolina','ND':'North Dakota','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico', \
                                'NV':'Nevada','NY':'New York','OH':'Ohio','OK':'Oklahoma','OR':'Oregon','PA':'Pennsylvania' ,\
                                'RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota','TN':'Tennessee','TX':'Texas', \
                                'UT':'Utah','VA':'Virginia','WA':'Washington','WI':'Wisconsin','WY':'Wyoming'})

In [48]:
df3p=pd.merge(df_all,df2p,how="inner", left_on=['State','County','year'], right_on=['State','County_Name','Year'])

In [49]:
df3p=df3p.T.drop_duplicates().T

In [3]:
#df3p

In [56]:
def nit(df,year):
    
    dfyear=df3p[(df3p['year']==year)]  
    df_ten_years=df3p[(df3p['year']>year-11)&(df3p['year']<year)]
    
    dfno2=df_ten_years.groupby(["State","County"]).agg(NO2_tenyears=pd.NamedAgg(column="avg(NO2 Mean)", aggfunc="mean"), SO2_tenyears=pd.NamedAgg(column="avg(SO2 Mean)", aggfunc="mean"), O3_tenyears=pd.NamedAgg(column="avg(O3 Mean)", aggfunc="mean")).reset_index()
    
    df_final=pd.merge(dfyear, dfno2, how="inner", on=['State','County'])
    
    return df_final

In [57]:
l=[]
for i in range(2010,2017):
    l.append(nit(df3p,i))

In [58]:
df4p=pd.concat(l,join="inner")

In [59]:
df4p

,State,County,year,avg(NO2 Mean),avg(O3 Mean),avg(SO2 Mean),Population,Area,Deaths,pollution_death,...,Crude Rate,pollution_Crude_Rate,resp_Crude_Rate,circ_Crude_Rate,canc_Crude_Rate,resp_circ_Crude_Rate,population_density,NO2_tenyears,SO2_tenyears,O3_tenyears
0,North Dakota,Cass,2010,5.418302,0.026455,0.281607,149778,1768.0,732,466,...,488.72331,311.127135,50.07411,162.90777,98.145255,212.98188,84.716063,5.134675,0.230227,0.025016
1,California,Humboldt,2010,2.634853,0.022844,1.285161,134623,4052.52,1035,639,...,768.81365,474.658862,94.33752,228.787057,151.534285,323.124578,33.219577,3.808521,0.203946,0.020634
2,Arkansas,Pulaski,2010,10.004749,0.02765,1.292494,382748,807.91,3234,2020,...,844.942364,527.762392,58.785415,267.800224,201.176753,326.585639,473.750789,9.639826,2.217690,0.024706
3,Pennsylvania,York,2010,11.458178,0.030882,2.165477,434972,910.34,3505,2283,...,805.798994,524.86137,71.268955,244.843346,208.749069,316.112301,477.812685,13.787552,4.373538,0.030389
4,California,Los Angeles,2010,20.447296,0.022612,0.780942,9818605,4752.32,56740,40346,...,577.8825,410.91377,53.398624,212.474175,145.040971,265.8728,2066.065627,27.253926,1.923875,0.021813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,Alabama,Jefferson,2016,9.524873,0.028348,0.835118,659521,1123.87,7014,4054,...,1063.499115,614.688539,92.946244,302.795514,218.946781,395.741758,586.830327,10.185320,0.982086,0.020253
52,New York,Queens,2016,17.874275,0.02405,0.836166,2333054,177.35,14393,10652,...,616.916711,456.568944,51.991938,252.973142,151.603863,304.96508,13155.083169,20.129922,2.905442,0.022157
53,California,Solano,2016,8.40495,0.023891,0.611533,440207,906.85,3148,2041,...,715.118115,463.645512,66.786762,204.222105,192.636646,271.008866,485.424271,9.409483,0.781707,0.023283
54,Colorado,Denver,2016,25.17045,0.02121,1.140453,693060,154.88,4505,2647,...,650.015872,381.929414,64.929443,171.269443,145.730528,236.198886,4474.819215,27.105770,2.196415,0.020553


In [60]:
df4p.to_csv('df3.csv')